# CIECAM02 color appearance model
This program is for calculate parameters from CIE XYZ to CIECAM02 color apperance model.

Reference:
- https://en.wikipedia.org/wiki/CIECAM02
- https://en.wikipedia.org/wiki/LMS_color_space
- http://blog.sina.com.cn/s/blog_4b892b790100i5la.html

## Model description and background parameters
Background parameter.
Three surroundings in CIECAM02 model: average, dim, and dark.

| Surround condition | Surround ratio | $F$ | $c$   | $N_c$ | Application                     |
| ------------------ | -------------- | --- | ----- | ----- | ------------------------------- | 
| Average            | $S_R > 0.2$    | 1.0 | 0.69  | 1.0   | Viewing surface colors          |
| Dim                | $0<S_R<0.2$    | 0.9 | 0.59  | 0.95  | Viewing television              |
| Dark               | $S_R = 0$      | 0.8 | 0.525 | 0.8   | Using a projector in a dark room|

- $S_R$ = $L_{sw} / L_{dw}$: ratio of the absolute luminance of the reference white (white point) measured in the surround field to the display area. The 0.2 coefficient derives from the "gray world" assumption (~18%–20% reflectivity). It tests whether the surround luminance is darker or brighter than medium gray.
- $F$: factor determining degree of adaptation.
- $c$: impact of surrounding.
- $N_c$: chromatic induction factor.

Adoped white in test illuminant: 
$X_w$, $Y_w$, $Z_w$ and $Y_w = 100$.

Background in test condition:
$Y_b$.

Reference white in reference illuminant:
$X_{wr} = Y_{wr} = Z_{wr} = 100$.

Luminance of test adapting field ($cd/m^2$):
$L_A$.

Hue circle:

| Index | Red   | Yellow | Green  | Blue   | Red    |
| ----- | ----- | ------ | ------ | ------ | ------ |
| i     | 1     | 2      | 3      | 4      | 5      |
|hi     | 20.14 | 90.00  | 164.25 | 237.53 | 380.14 |
|ei     | 0.8   | 0.7    | 1.0    | 1.2    | 0.8    |
|Hi     | 0.0   | 100.0  | 200.0  | 300.0  | 400.0  |

In [1]:
function XYZ2CIECAM02(X,Y,Z,X_w,Y_w,Z_w,Y_b,L_A,surround_cond)
#     The surrounding conditon parameter:
    if surround_cond == "average"
        F = 1; c = 0.69; N_c = 1.0;
    elseif surround_cond == "dim"
        F = 0.9; c = 0.59; N_c = 0.95;
    elseif surround_cond == "dark"
        F = 0.8; c = 0.525; N_c = 0.8;
    elseif surround_cond == "light_box"
        F = 0.8; c = 0.41; N_c = 0.8;
    end
    
#     Color matrix:
    M_cat02 = [0.7328 0.4296 -0.1624; -0.7036 1.6975 0.0061; 0.0030 0.0136 0.9834];
    M_cat02_inv = [1.0961 -0.2789 0.1827; 0.4544 0.4735 0.0721; -0.0096 -0.0057 1.0153];
#     M_cat02_inv = M_cat02^(-1);
    M_hpe = [0.38971 0.68898 -0.07868; -0.22981 1.1834 0.04641; 0 0 1];
    
    
#     Step 1:
    RGB_w = M_cat02 * [X_w; Y_w; Z_w];
    RGB = M_cat02 * [X; Y; Z];
    
#     Step 2:
    D = F * (1 - (1/3.6)*exp((-1*L_A-42)/92))
    RGB_w = M_cat02 * [X_w; Y_w; Z_w];
    D_RGB = Y_w*D./RGB_w + 1 - D;
    
#     Step 3:
    RGB_c = D_RGB .* RGB;
    RGB_wc = M_cat02 * [X_w; Y_w; Z_w];
    
#     Step 4:
    k = 1/(5*L_A + 1);
    n = Y_b/Y_w;
    z = 1.48 + sqrt(n);
    N_bb = N_cb = 0.725 * (1/n)^0.2;
    F_L = 0.2*k^4*(5*L_A) + 0.1*(1-k^4)^2*(5*L_A)^(1/3);
    
#     Step 5:
    RGB_apo = M_hpe * M_cat02_inv * RGB_c;
    RGB_w_apo = M_hpe * M_cat02_inv * RGB_wc;
    
#     Step 6:
    RGB_a_apo = [step6(RGB_apo[1],F_L), step6(RGB_apo[2],F_L), step6(RGB_apo[3],F_L)];
    RGB_aw_apo = [step6(RGB_w_apo[1],F_L), step6(RGB_w_apo[2],F_L), step6(RGB_w_apo[3],F_L)];
    
#     Step 7:
    C_1 = RGB_a_apo[1] - RGB_a_apo[2];
    C_2 = RGB_a_apo[2] - RGB_a_apo[3];
    C_3 = RGB_a_apo[3] - RGB_a_apo[1];
    a = C_1 - (1/11)*C_2;
    b = 1/2*(C_2 - C_3)/4.5;
    h = cal_h(a,b);
    if h < 20.14
        h_apo = h + 360;
    else
        h_apo = h;
    end
    ei, Hi, hi, hi1, ei1 = cal_hue(h_apo);
    H = Hi + (100*(h_apo-hi)/ei) / ((h_apo-hi)/ei + (hi1-h_apo)/ei1);
    # Verison 1: the textbook
    et = ei + (ei1-ei)*(h_apo-hi)/(hi1-hi);
    # Verison 2: the blog and wiki page
#     et = 1/4*(cos(pi/1.8*h+2)+3.8)
    A = (2*RGB_a_apo[1] + RGB_a_apo[2] + (1/20)*RGB_a_apo[3]) * N_bb;
    A_w = (2*RGB_aw_apo[1] + RGB_aw_apo[2] + (1/20)*RGB_aw_apo[3]) * N_bb;
    J = 100 * (A/A_w)^(c*z);
    Q = (4/c)*sqrt(1/100*J) * (A_w+4) * F_L^(1/4);
    t = (50000/13 * N_c*N_cb*et*sqrt(a^2+b^2)) / (RGB_a_apo[1] + RGB_a_apo[2] + 21/20*RGB_a_apo[3]);
    C = t^0.9 * sqrt(1/100*J) * (1.64-0.29^n)^0.73;
    M = C*F_L^(1/4);
    s = 100 * sqrt(M/Q);
    
    return J, C, H, h, M, s, Q
end

function step6(R, F_L)
    if R >= 0
        R_a = (400*(F_L*abs(R)/100)^0.42)  /  (27.13 + (F_L*abs(R)/100)^0.42) + 0.1;
    else
        R_a = -1 *( (400*(F_L*abs(R)/100)^0.42)  /  (27.13 + (F_L*abs(R)/100)^0.42) + 0.1);
    end
    return R_a
end;

function cal_h(a,b)
    h = atan2(b,a);
    if h < 0
        h = h + 2*pi;
    end
    h = h / pi * 180;
    return h
end;

function cal_hue(h)
    if 20.14 < h <= 90
        ei = 0.8; Hi = 0; hi = 20.14; hi1 = 90; ei1 = 0.7;
    elseif 90 < h <= 164.25
        ei = 0.7; Hi = 100; hi = 90; hi1 = 164.25; ei1 = 1;
    elseif 164.25 < h <= 237.53
        ei = 1; Hi = 200; hi = 164.25; hi1 = 237.53; ei1 = 1.2;
    elseif 237.53 < h <= 380.14
        ei = 1.2; Hi = 300; hi = 237.53; hi1 = 380.14; ei1 = 0.8;
    end
    return ei, Hi, hi, hi1, ei1
end;

### Test data

In [2]:
X = 57.06; Y = 43.06; Z = 31.96;
X_w = 95.05; Y_w = 100.00; Z_w = 108.88;
L_A = 31.83;
F = 1.0; c = 0.69; N_c = 1.0;
Y_b = 20;

In [3]:
# X = 3.53; Y = 6.56; Z = 2.14;
# X_w = 109.85; Y_w = 100; Z_w = 35.58;
# L_A = 318.31;

In [4]:
# X = 19.01; Y = 20; Z = 21.78;
# X_w = 109.85; Y_w = 100; Z_w = 35.58;
# L_A = 31.83;

In [5]:
J, C, H, h, M, s, Q = XYZ2CIECAM02(X,Y,Z,X_w,Y_w,Z_w,Y_b,L_A,"average")

(66.5870997970122,50.24654606964671,399.3931835332832,19.561910042859314,43.11119985462551,52.8920855497015,154.10216021199972)

## Import real data

In [6]:
using DataFrames
data = readtable("QJ_csv.csv", header=true, separator=',');

In [7]:
head(data)

,_name,L_,a_,b_,C_,h_,X,Y,Z
1,QJ01,30.95,5.61,-14.6,15.64,291.0,6.824,6.632,11.703
2,QJB1,27.12,6.03,-15.31,16.45,291.51,5.361,5.138,9.668
3,QJ2,28.77,7.16,-18.78,20.09,290.87,6.08,5.749,11.855
4,QJB2,26.37,6.59,-17.22,18.43,290.94,5.138,4.872,9.864
5,QJ3,26.37,8.68,-22.66,24.26,290.96,5.312,4.874,11.762
6,QJB3,24.62,6.33,-13.95,15.32,294.4,4.529,4.295,7.947


## Different Environment
Background parameters:

In [8]:
Y_b = 20;
X_w,Y_w,Z_w = 95.02,100,108.81;
L_A = 63.6619;

In [9]:
data1 = DataFrame(name=[], environment=[], J=[], C=[], H=[], h=[], M=[], s=[], Q=[]);

for i = 1:size(data)[1]
    J, C, H, h, M, s, Q = XYZ2CIECAM02(data[:X][i],data[:Y][i],data[:Z][i],X_w,Y_w,Z_w,Y_b,L_A,"average");
    push!(data1, @data([data[:_name][i],"Average",J,C,H,h,M,s,Q]));
end

for i = 1:size(data)[1]
    J, C, H, h, M, s, Q = XYZ2CIECAM02(data[:X][i],data[:Y][i],data[:Z][i],X_w,Y_w,Z_w,Y_b,L_A,"dim");
    push!(data1, @data([data[:_name][i],"Dim",J,C,H,h,M,s,Q]));
end

for i = 1:size(data)[1]
    J, C, H, h, M, s, Q = XYZ2CIECAM02(data[:X][i],data[:Y][i],data[:Z][i],X_w,Y_w,Z_w,Y_b,L_A,"dark");
    push!(data1, @data([data[:_name][i],"Dark",J,C,H,h,M,s,Q]));
end

for i = 1:size(data)[1]
    J, C, H, h, M, s, Q = XYZ2CIECAM02(data[:X][i],data[:Y][i],data[:Z][i],X_w,Y_w,Z_w,Y_b,L_A,"light_box");
    push!(data1, @data([data[:_name][i],"Light Box",J,C,H,h,M,s,Q]));
end

In [10]:
head(data1)

,name,environment,J,C,H,h,M,s,Q
1,QJ01,Average,23.15457059550678,18.325826193737015,318.831810215608,274.3473774946405,16.658441356007753,39.886250394276495,104.70994746253159
2,QJB1,Average,20.16792827714322,19.143969175165427,318.8562496254963,274.39104139626204,17.40214517229803,42.19891489168987,97.72377162345116
3,QJ2,Average,21.386776871013687,22.827760511759532,317.79190713513816,272.48038787140524,20.75076483613587,45.409417582939234,100.63342299119621
4,QJB2,Average,19.570445046199325,21.223649605537513,317.990002086034,272.8374109767213,19.292604795909995,44.76725551499718,96.26533510727641
5,QJ3,Average,19.50390628441435,26.842204819093336,316.69241313274506,270.486922705953,24.399952838013466,50.3882534387635,96.10154654832223
6,QJB3,Average,18.348369254751965,17.637309321338915,321.15475533066194,278.4544601572548,16.032569549726766,41.47318704361156,93.21124564774327


In [11]:
writetable("data1.csv", data1, separator = ',', header = true)

## Changing Illuminance
Background parameters:

In [12]:
Y_b = 20;
X_w,Y_w,Z_w = 95.02,100,108.81;
L_A1 = 63.6619;
L_A2 = 143.38;
L_A3 = 149.81;

In [13]:
data2 = DataFrame(name=[], L_A=[], J=[], C=[], H=[], h=[], M=[], s=[], Q=[]);

for i = 1:size(data)[1]
    J, C, H, h, M, s, Q = XYZ2CIECAM02(data[:X][i],data[:Y][i],data[:Z][i],X_w,Y_w,Z_w,Y_b,L_A,"average");
    push!(data2, @data([data[:_name][i],L_A1,J,C,H,h,M,s,Q]));
end

for i = 1:size(data)[1]
    J, C, H, h, M, s, Q = XYZ2CIECAM02(data[:X][i],data[:Y][i],data[:Z][i],X_w,Y_w,Z_w,Y_b,L_A,"average");
    push!(data2, @data([data[:_name][i],L_A2,J,C,H,h,M,s,Q]));
end

for i = 1:size(data)[1]
    J, C, H, h, M, s, Q = XYZ2CIECAM02(data[:X][i],data[:Y][i],data[:Z][i],X_w,Y_w,Z_w,Y_b,L_A,"average");
    push!(data2, @data([data[:_name][i],L_A3,J,C,H,h,M,s,Q]));
end

In [14]:
head(data2)

,name,L_A,J,C,H,h,M,s,Q
1,QJ01,63.6619,23.15457059550678,18.325826193737015,318.831810215608,274.3473774946405,16.658441356007753,39.886250394276495,104.70994746253159
2,QJB1,63.6619,20.16792827714322,19.143969175165427,318.8562496254963,274.39104139626204,17.40214517229803,42.19891489168987,97.72377162345116
3,QJ2,63.6619,21.386776871013687,22.827760511759532,317.79190713513816,272.48038787140524,20.75076483613587,45.409417582939234,100.63342299119621
4,QJB2,63.6619,19.570445046199325,21.223649605537513,317.990002086034,272.8374109767213,19.292604795909995,44.76725551499718,96.26533510727641
5,QJ3,63.6619,19.50390628441435,26.842204819093336,316.69241313274506,270.486922705953,24.399952838013466,50.3882534387635,96.10154654832223
6,QJB3,63.6619,18.348369254751965,17.637309321338915,321.15475533066194,278.4544601572548,16.032569549726766,41.47318704361156,93.21124564774327


In [15]:
writetable("data2.csv", data2, separator = ',', header = true)

## Changing Light Source
Background parameters:

In [16]:
Y_b = 20;
L_A1 = 63.6619;
X_w1,Y_w1,Z_w1 = 95.04,100,108.88;
X_w2,Y_w2,Z_w2 = 109.85,100,35.58;
X_w3,Y_w3,Z_w3 = 96.42,100,82.51;

In [17]:
data3 = DataFrame(name=[], Light_Source=[], J=[], C=[], H=[], h=[], M=[], s=[], Q=[]);

for i = 1:size(data)[1]
    J, C, H, h, M, s, Q = XYZ2CIECAM02(data[:X][i],data[:Y][i],data[:Z][i],X_w1,Y_w1,Z_w1,Y_b,L_A,"average");
    push!(data3, @data([data[:_name][i],"D65",J,C,H,h,M,s,Q]));
end

for i = 1:size(data)[1]
    J, C, H, h, M, s, Q = XYZ2CIECAM02(data[:X][i],data[:Y][i],data[:Z][i],X_w2,Y_w2,Z_w2,Y_b,L_A,"average");
    push!(data3, @data([data[:_name][i],"A",J,C,H,h,M,s,Q]));
end

for i = 1:size(data)[1]
    J, C, H, h, M, s, Q = XYZ2CIECAM02(data[:X][i],data[:Y][i],data[:Z][i],X_w3,Y_w3,Z_w3,Y_b,L_A,"average");
    push!(data3, @data([data[:_name][i],"D50",J,C,H,h,M,s,Q]));
end

In [18]:
head(data3)

,name,Light_Source,J,C,H,h,M,s,Q
1,QJ01,D65,23.15376965773379,18.30752564395431,318.8147988582711,274.3169789316577,16.641805891275357,39.86651561344643,104.70897139049355
2,QJB1,D65,20.167187017729407,19.12707387114935,318.84200434008693,274.365591691493,17.386787096314535,42.18050909377864,97.72275496123223
3,QJ2,D65,21.385920870832965,22.810841964334365,317.7796783206952,272.4583268053215,20.735385631556962,45.39286035037101,100.63221149969108
4,QJB2,D65,19.569675601311516,21.207387715466098,317.9773981953913,272.81471453283393,19.277822502373613,44.75036299133176,96.26421028354362
5,QJ3,D65,19.503000922996584,26.826596956809617,316.68316115858244,270.47006234869633,24.385765065205995,50.373985503453156,96.10008233266775
6,QJB3,D65,18.34770134207998,17.620695520694216,321.142471070197,278.4329679110853,16.017467364383837,41.45386118808924,93.21029236577449


In [19]:
writetable("data3.csv", data3, separator = ',', header = true)